In [1]:
import sys
import os
sys.path.append("./../")
sys.path.append("./../external")
sys.path.append("./../external")
import numpy as np
import time
from scipy.optimize import differential_evolution, minimize
import matplotlib.pyplot as plt
from pyminion import *
import concurrent.futures
import threading
from cec_2011 import * #commmet this line if you do not have matlab installed

In [ ]:
p = Problem10()
print(p.evaluate(np.random.uniform(p.lb, p.ub)))
p.eng.quit()

# CEC2011
-------------

In [2]:
results=[]
for j in range(5):
    for i in range(1, 23):
        result = {}
        if i ==10 : continue
        func = CEC2011(function_number=i, max_workers=8)
        bounds = func.getBounds()
        dimension = func.dimension
        Nmaxeval=50000
        if (dimension<10): Nmaxeval=20000
        if (i==3 or i==4):  Nmaxeval=200
        
        result['Dimensions'] = dimension
        result['Function_number'] = i
        print("Function : ", i)
        #result =minimize(objective_function, initial_guess, args=(), method='Nelder-Mead', options={"maxfev":Nmaxeval, "adaptive":True }  ) 
        #print("SIMPLEX done")
        result_arrde = ARRDE(func.evaluate, bounds, data=None,  x0=None, population_size=0, maxevals=Nmaxeval, tol=0.0 ).optimize()
        print("\tObj ARRDE :  ", result_arrde.fun)
        result_lsrtde = LSRTDE(func.evaluate, bounds, data=None,  x0=None, populationSize=0, maxevals=Nmaxeval ).optimize()
        print("\tObj LSRTDE :  ", result_lsrtde.fun)
        #result_lshade = LSHADE(objective_function_vect, bounds, data=None,  x0=None, population_size=0, maxevals=Nmaxeval, options= {}).optimize()
        #print("LSHADE done")
        result_nlshadersp = NLSHADE_RSP (func.evaluate, bounds, data=None, x0=None, population_size=0, 
                            maxevals=Nmaxeval, callback=None, seed=None, memory_size=20*dimension, archive_size_ratio=2.1).optimize()
        print("\tObj NLSHADE RSP :  ", result_nlshadersp.fun)
        #result_j20 = j2020(func.evaluate, bounds, data=None,  x0=None, populationSize=0, maxevals=Nmaxeval ).optimize()
        #print("\tObj j20 :  ", result_j20.fun)


        result["ARRDE"] = result_arrde.fun
        result["LSRTDE"] = result_lsrtde.fun
        result["NLSHADE_RSP"] = result_nlshadersp.fun

        results.append(result)
        
        del func

Function :  1
	Obj ARRDE :   4.702488316458331e-28
	Obj LSRTDE :   6.279548961349509
	Obj NLSHADE RSP :   10.196258280000064
Function :  2
	Obj ARRDE :   -23.81113218827846
	Obj LSRTDE :   -12.277344914710234
	Obj NLSHADE RSP :   -21.68974354419291
Function :  3
	Obj ARRDE :   -0.040486847549302776
	Obj LSRTDE :   -0.040487564556161405
	Obj NLSHADE RSP :   -0.04047435894679451
Function :  4


# Evaluate CEC Problem and Repeat for N times using multithreading

In [ ]:
class FunctionEvaluator:
    def __init__(self, func):
        self.func = func
        self.n_calls = 0

    def __call__(self, X):
        self.n_calls += 1
        ret = self.func(np.array([X]))[0]
        #print(X.shape, ret.shape)
        return ret
    
class VectorizedEvaluator : 
    def __init__(self, func):
        self.func = func
        self.n_calls = 0

    def __call__(self, X):
        self.n_calls += X.shape[0]
        ret = self.func(np.array(X))
        return ret
    
class PyminionFunc : 
    def __init__(self, func):
        self.func = func
        self.n_calls = 0

    def __call__(self, X, data=None):
        X= np.array(X)
        self.n_calls += X.shape[0]
        return  self.func(X)
    
class VectorizedEvaluatorDE : 
    def __init__(self, func):
        self.func = func
        self.n_calls = 0

    def __call__(self, X):
        X = np.array(X).T
        self.n_calls += X.shape[0]
        return self.func(X)

goptimum_cec22 = {
    1 : 300, 2: 400, 3: 600, 4: 800, 5:900, 6:1800, 7:2000, 8:2200, 9:2300, 10: 2400, 11:2600, 12: 2700
}
goptimum_cec20 = {
    1 : 100, 2: 1100, 3: 700, 4: 1900, 5:1700, 6:1600, 7:2100, 8:2200, 9:2400, 10: 2500
}

In [ ]:
# Global results variable
results = []
results_lock = threading.Lock()

def test_optimization_threadsafe(func, bounds, dimension, func_name, Nmaxeval):
    global results
    result = {}
    result['Dimensions'] = dimension
    result['Function'] = func_name
    # Initialize bounds
    bounds_list = [bounds] * dimension
    # Create wrapped function evaluator
    evaluator = FunctionEvaluator(func)
    vecEvaluator = VectorizedEvaluator(func)
    vecEvaluatorDE = VectorizedEvaluatorDE(func)
    pyminionFunc = PyminionFunc(func)
    popsize= int(np.ceil((np.log10(Nmaxeval))**2.0+dimension/2.0))

     #----------------------------------------------------------------#
    options_LSHADE= {
        "memory_size": 6, 
        "archive_size_ratio": 2.6, 
        "population_reduction" : True, 
        "reduction_strategy": "linear",
        "minimum_population_size": 4, 
    }
    
    options_JADE =  {
        "mutation_strategy": "current_to_pbest_A1_1bin",
        "archive_size_ratio": 1.0, 
        "population_reduction" : True, 
        "reduction_strategy": "linear",
        "minimum_population_size": dimension, 
        "c" : 0.1,
    }

    #pyminionFunc.n_calls =0
    #jade = JADE (pyminionFunc, bounds_list, options=options_JADE, data=None, x0=None, population_size=0, 
    #                 maxevals=Nmaxeval, tol=0.0, callback=None, boundStrategy="reflect-random", seed=None)
    #res = jade.optimize()
    #result['JADE'] = res.fun
    
    
    #pyminionFunc.n_calls =0
    lshade = LSHADE (pyminionFunc, bounds_list, options=options_LSHADE, data=None, x0=None, population_size=0, 
                     maxevals=Nmaxeval, tol=0.0, callback=None, boundStrategy="reflect-random", seed=None)
    res = lshade.optimize()
    result['LSHADE'] = res.fun
    #print(pyminionFunc.n_calls)
    #----------------------------------------------------------------#

    
    #print(pyminionFunc.n_calls)
    #----------------------------------------------------------------#

     #----------------------------------------------------------------#
    pyminionFunc.n_calls =0
    lshade_rsp = NLSHADE_RSP (pyminionFunc, bounds_list, data=None, x0=None, population_size=0, 
                     maxevals=Nmaxeval, callback=None, seed=None, memory_size=20*dimension, archive_size_ratio=2.1)
    res = lshade_rsp.optimize()
    result['NLSHADE_RSP'] = res.fun
    #print(pyminionFunc.n_calls)
    #----------------------------------------------------------------#

     #----------------------------------------------------------------#
    pyminionFunc.n_calls =0
    j20 = j2020 (pyminionFunc, bounds_list, data=None, x0=None,  
                     maxevals=Nmaxeval, callback=None, seed=None, populationSize=0)
    res = j20.optimize()
    result['j2020'] = res.fun
    #print(pyminionFunc.n_calls)
    #----------------------------------------------------------------#

     #----------------------------------------------------------------#
    pyminionFunc.n_calls =0
    lsrtde =LSRTDE (pyminionFunc, bounds_list, data=None, x0=None,  
                     maxevals=Nmaxeval, callback=None, seed=None, populationSize=0)
    res = lsrtde.optimize()
    result['LSRTDE'] = res.fun
    #print(pyminionFunc.n_calls)
    #----------------------------------------------------------------#

    #----------------------------------------------------------------
    
    pyminionFunc.n_calls=0
    arrde = ARRDE (pyminionFunc, bounds_list, data=None, x0=None, population_size=0, 
                       maxevals=Nmaxeval, tol=0., callback=None, boundStrategy="reflect-random", seed=None)
    res = arrde.optimize()
    result['ARRDE'] = res.fun
    #print(pyminionFunc.n_calls)
    #----------------------------------------------------------------#

    #Differential Evolution (DE)
    #vecEvaluatorDE.n_calls = 0
    #de_result = differential_evolution(vecEvaluatorDE, bounds_list, popsize=3, strategy='best1bin',
    #                                     maxiter=int(Nmaxeval/(3*dimension)), vectorized=True, updating="deferred", disp=False,polish=False)
    #result['Scipy DE'] = de_result.fun

    with results_lock:
        results.append(result)
    print(result)

Nmaxeval = 20000
dimension = 20

def run_test_optimization(j, dim):
    cec_func = CEC2020Functions(function_number=j, dimension=dim)
    test_optimization_threadsafe(cec_func, (-100, 100), dim, "func_" + str(j), Nmaxeval)


func_numbers = [1,2,3,4,5,6,7, 8, 9,10, 11, 12] #2022
func_numbers = [1,2,3,4,5,6,7, 8, 9,10]
#func_numbers = [ 2,5,11, 10]
#func_numbers = [ 9, 10]
#func_numbers= [2, 8, 10, 11]
#func_numbers= range(20, 30)
Nrepeat= 11

with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:

    futures = []
    for k in range(Nrepeat):
        for j in func_numbers:
            futures.append(executor.submit(run_test_optimization, j, dimension))
    concurrent.futures.wait(futures)
    for f in futures: f.result()

for num in func_numbers : 
    mydict= {}
    #algoRes = {"NLSHADE_RSP" : [],  "ARRDE":[]}
    algoRes = {"NLSHADE_RSP" : [],  "ARRDE":[], "j2020":[], "LSHADE": [], 'Scipy DE':[],"LSRTDE":[] }
    #algoRes = {"NLSHADE_RSP" : [],  "ARRDE":[], "j2020":[]}
    for res in list(results) : 
        for algo in algoRes.keys() : 
            if res['Function'] == "func_"+str(num) :
                algoRes[algo].append(res[algo])

    full_results= {}
    for key, val in algoRes.items() : 
        error = np.abs(np.array(val)-goptimum_cec22[num])
        #error = error/goptimum_cec20[num]
        error = val
        full_results[key] = (np.min(error), np.mean(error), np.std(error))

    print("Full results for function "+str(num) +":\n\t", full_results)


# Curve Fitting

## Fit Polynomial

In [ ]:
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt

dimension=21
# Step 1: Generate data points from a polynomial 
np.random.seed(8)  # For reproducibility

# True coefficients of the polynomial (degree 10)
true_coefficients = [np.random.uniform(-1.0, 1.0)*1.0**i for i in range(dimension)]  # Random coefficients for the polynomial
print(true_coefficients)

# Generate 50 data points
x_data = np.linspace(1e-8, 1, dimension+10)
y_data = np.polyval(true_coefficients, x_data)
#y_data = y_data + np.random.normal(0, np.sqrt(np.abs(y_data)))  # Add some noise

# Step 2: Define the polynomial model
def polynomial_model(x, coefficients):
    """Given x and coefficients, return the polynomial value."""
    return np.polyval(coefficients, x)

# Step 3: Define the objective function to minimize
def objective_function(coefficients, x, y):
    """Objective function for optimization: Sum of squared errors."""
    y_pred = polynomial_model(x, coefficients)
    return np.sum((y - y_pred)**2)

def objective_function_vect(coefficients, data) : 
    ret = []
    for coeff in coefficients : 
        ret.append(objective_function(coeff, x_data, y_data))
    return ret

bounds= [(-10, 10)]*dimension
Nmaxeval= 100000#1000*dimension

# Optimize the coefficients using Nelder-Mead
#result = opt.minimize(objective_function, np.ones(dimension), args=(x_data, y_data), method='Nelder-Mead', options={"maxfev":Nmaxeval, "adaptive":True }  ) 

result_arrde = ARRDE(objective_function_vect, bounds, data=None,  x0=None, population_size=0, maxevals=int(1.0*Nmaxeval), tol=0.0 ).optimize()
#result_arrde = opt.minimize(objective_function, result_arrde.x, args=(x_data, y_data), method='Nelder-Mead', options={"maxfev":int(0.05*Nmaxeval), "adaptive":True }  ) 

result_lsrtde = LSRTDE(objective_function_vect, bounds, data=None,  x0=None, populationSize=0, maxevals=Nmaxeval ).optimize()
result_lshade = LSHADE(objective_function_vect, bounds, data=None,  x0=None, population_size=0, maxevals=Nmaxeval, options= {}).optimize()
result_nlshadersp = NLSHADE_RSP (objective_function_vect, bounds, data=None, x0=None, population_size=0, 
                     maxevals=Nmaxeval, callback=None, seed=None, memory_size=20*dimension, archive_size_ratio=2.1).optimize()
result_j20 = j2020(objective_function_vect, bounds, data=None,  x0=None, populationSize=0, maxevals=Nmaxeval ).optimize()
#result_de = differential_evolution(objective_function, bounds, args=(x_data, y_data), popsize=3, strategy='best1exp',
#                                         maxiter=int(Nmaxeval/(3*dimension)), vectorized=False, disp=False,polish=False)

# Step 5: Extract the fitted coefficients

#fitted_coefficients_nm = result.x
fitted_coefficients_arrde = result_arrde.x
fitted_coefficients_lsrtde = result_lsrtde.x
fitted_coefficients_nlshade = result_nlshadersp.x
fitted_coefficients_j20= result_j20.x
#fitted_coefficients_de= result_de.x
fitted_coefficients_lshade= result_lshade.x

# Step 6: Plot the results
plt.scatter(x_data, y_data, label='Data Points')
#plt.plot(x_data, np.polyval(true_coefficients, x_data), 'r-', label='True Polynomial')
#plt.plot(x_data, np.polyval(fitted_coefficients_nm, x_data), 'g--', label='Fitted nm')
plt.plot(x_data, np.polyval(fitted_coefficients_arrde, x_data), 'b--', label='Fitted arrde')
plt.plot(x_data, np.polyval(fitted_coefficients_lsrtde, x_data), 'g--', label='Fitted LSRTDE')
plt.plot(x_data, np.polyval(fitted_coefficients_nlshade, x_data), 'black', label='Fitted NLSHADE RSP')
plt.plot(x_data, np.polyval(fitted_coefficients_j20, x_data), 'yellow', label='Fitted j20')
plt.legend()
plt.title('Polynomial Fit using Nelder-Mead')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

# Display the true and fitted coefficients
#print("True coefficients:    ", true_coefficients)
#print("Fitted coefficients NM :  ", fitted_coefficients_nm)
#print("Fitted coefficients ARRDE :  ", fitted_coefficients_arrde)

print("")

#print("Obj NM :  ",result.fun)
print("Obj ARRDE :  ", result_arrde.fun)
print("Obj LSRTDE :  ", result_lsrtde.fun)
print("Obj NLSHADE RSP :  ", result_nlshadersp.fun)
print("Obj lshade:  ", result_lshade.fun)
print("Obj j20 :  ", result_j20.fun)
#print("Obj de :  ", result_de.fun)

## Fit RBF basis functions

In [ ]:
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt


np.random.seed(5)  # For reproducibility
num_rbf = 20
true_centers = 10*(-1+2*np.random.random(num_rbf))
true_widths = np.random.rand(num_rbf) + 1.0  # Widths (variances)
true_coeffs = 2.0*np.random.rand(num_rbf)
print(true_centers)
print(true_widths)
print(true_coeffs)
dimension= num_rbf*3

# Define a Gaussian RBF function
def rbf(x, center, width):
    """Compute a Gaussian RBF value given x, center, and width."""
    return (1.0/2.0*np.pi*width**2)*np.exp(-((x - center) ** 2) / (2 * width ** 2))

# Define the function as a sum of RBFs
def rbf_sum(x, centers, widths, coeffs):
    """Sum of Gaussian RBFs."""
    result = np.zeros_like(x)
    coeffs= np.array(coeffs)
    norm_coeff = coeffs/np.sum(coeffs)
    for i in range(len(centers)):
        result += norm_coeff[i] * rbf(x, centers[i], widths[i])
    return result


# Generate 50 data points
x_data = np.linspace(-20, 20,dimension+50)
y_data = rbf_sum(x_data, true_centers, true_widths, true_coeffs)  # Add some noise


# Step 2: Define the model for fitting
def rbf_model(x, params):
    """RBF model with combined parameters (centers, widths, and coefficients)."""
    num_rbf = len(params) // 3
    centers = params[:num_rbf]
    widths = params[num_rbf:2*num_rbf]
    coeffs = params[2*num_rbf:]
    return rbf_sum(x, centers, widths, coeffs)

def rbf_data(params):
    """RBF model with combined parameters (centers, widths, and coefficients)."""
    num_rbf = len(params) // 3
    centers = params[:num_rbf]
    widths = params[num_rbf:2*num_rbf]
    coeffs = params[2*num_rbf:]
    return rbf_sum(x_data, centers, widths, coeffs)

# Step 3: Define the objective function to minimize
def objective_function(params):
    """Objective function for optimization: Sum of squared errors."""
    y_pred = rbf_model(x_data, params)
    return np.sum((y_data - y_pred) ** 2)




executor = concurrent.futures.ThreadPoolExecutor(max_workers=4)
def objective_function_vect(params, data) : 
    #ret = []
    #for p in params : 
    #    ret.append(objective_function(p, x_data, y_data))
    ret = list(executor.map(objective_function, params))
    return ret


initial_guess = np.concatenate([
    np.linspace(-100, 100, num_rbf),  # Initial guess for centers
    np.ones(num_rbf),  # Initial guess for widths
    np.zeros(num_rbf)  # Initial guess for coefficients
])


bounds= [(-10, 10)]*dimension
Nmaxeval=500000#5000*dimension

# Optimize the coefficients using Nelder-Mead

#result = opt.minimize(objective_function, initial_guess, args=(), method='Nelder-Mead', options={"maxfev":Nmaxeval, "adaptive":True }  ) 
#print("SIMPLEX done")
result_arrde = ARRDE(objective_function_vect, bounds, data=None,  x0=None, population_size=0, maxevals=Nmaxeval, tol=0.0 ).optimize()
print("ARRDE done")
result_lsrtde = LSRTDE(objective_function_vect, bounds, data=None,  x0=None, populationSize=0, maxevals=Nmaxeval ).optimize()
print("LSRTDE done")
result_lshade = LSHADE(objective_function_vect, bounds, data=None,  x0=None, population_size=0, maxevals=Nmaxeval, options= {}).optimize()
print("LSHADE done")
result_nlshadersp = NLSHADE_RSP (objective_function_vect, bounds, data=None, x0=None, population_size=0, 
                     maxevals=Nmaxeval, callback=None, seed=None, memory_size=20*dimension, archive_size_ratio=2.1).optimize()
print("RSP done")
result_j20 = j2020(objective_function_vect, bounds, data=None,  x0=None, populationSize=0, maxevals=Nmaxeval ).optimize()
print("j20 done")
#result_de = differential_evolution(objective_function, bounds, args=(x_data, y_data), popsize=1, strategy='best1bin',
#                                         maxiter=int(Nmaxeval/(3*dimension)), vectorized=False, disp=False,polish=False)
#print("DE done")

plt.plot(x_data, y_data, label="data", color="black")
plt.plot(x_data, rbf_data(result_arrde.x), label="ARRDE")
plt.plot(x_data, rbf_data(result_lsrtde.x), label="LSRTDE")
plt.plot(x_data, rbf_data(result_nlshadersp.x), label="NLSHADE RSP")
#plt.plot(x_data, rbf_data(result_j20.x), label="j2020")
plt.legend()


print("")
#
#print("Obj NM :  ",result.fun)
print("Obj ARRDE :  ", result_arrde.fun)
print("Obj LSRTDE :  ", result_lsrtde.fun)
print("Obj NLSHADE RSP :  ", result_nlshadersp.fun)
print("Obj lshade:  ", result_lshade.fun)
#print("Obj j20 :  ", result_j20.fun)
#print("Obj de :  ", result_de.fun)

executor.shutdown()


## Fit NN

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import scipy.optimize as opt
from scipy.optimize import differential_evolution, minimize
import matplotlib.pyplot as plt

# Fix the random seed for reproducibility
np.random.seed(1)
torch.manual_seed(1)

# Generate data points from a polynomial of degree 10
def generate_polynomial_data(x, coefficients):
    """Generates data points based on a polynomial function."""
    y = np.zeros_like(x)
    for i, coeff in enumerate(coefficients):
        y += coeff * x ** i
    return y

# True polynomial coefficients
true_coeffs = np.random.randn(11)  # Degree 10 polynomial has 11 coefficients
print("True coefficients: ", true_coeffs)

# Generate 50 data points
x_data = np.linspace(-10, 10, 50)
y_data = generate_polynomial_data(x_data, true_coeffs)

# Convert data to PyTorch tensors
x_tensor = torch.tensor(x_data, dtype=torch.float32).unsqueeze(1)  # (50, 1)
y_tensor = torch.tensor(y_data, dtype=torch.float32).unsqueeze(1)  # (50, 1)

# Define a simple neural network with a single hidden layer
class SimpleNN(nn.Module):
    def __init__(self, num_inputs, num_hidden, num_outputs):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(num_inputs, num_hidden)
        self.fc2 = nn.Linear(num_hidden, num_outputs)
    
    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the neural network
num_hidden = 10
nn = SimpleNN(num_inputs=1, num_hidden=num_hidden, num_outputs=1)

# Define the objective function for optimization
def nn_objective_function(params, x, y, nn_model):
    """Objective function: Mean Squared Error between predictions and true values."""
    # Update the network weights and biases from flattened params
    with torch.no_grad():
        start = 0
        
        # Extract weights and biases for the first layer
        nn_model.fc1.weight.data = torch.tensor(params[start:start + 1 * num_hidden].reshape(1, num_hidden), dtype=torch.float32)
        start += 1 * num_hidden
        nn_model.fc1.bias.data = torch.tensor(params[start:start + num_hidden], dtype=torch.float32)
        start += num_hidden
        
        # Extract weights and biases for the second layer
        nn_model.fc2.weight.data = torch.tensor(params[start:start + num_hidden * 1].reshape(num_hidden, 1), dtype=torch.float32)
        start += num_hidden * 1
        nn_model.fc2.bias.data = torch.tensor(params[start:start + 1], dtype=torch.float32)
    
    # Forward pass
    with torch.no_grad():
        y_pred = nn_model(x).detach()
    loss = torch.mean((y - y_pred) ** 2).item()
    return loss

# Flatten the network parameters into a single vector
def flatten_params(nn_model):
    """Flatten the network parameters into a single vector."""
    with torch.no_grad():
        fc1_weight = nn_model.fc1.weight.data.numpy().flatten()
        fc1_bias = nn_model.fc1.bias.data.numpy()
        fc2_weight = nn_model.fc2.weight.data.numpy().flatten()
        fc2_bias = nn_model.fc2.bias.data.numpy()
    return np.concatenate([fc1_weight, fc1_bias, fc2_weight, fc2_bias])

# Unflatten parameters back into network weights
def unflatten_params(nn_model, params):
    """Unflatten the parameter vector back into weights and biases."""
    start = 0
    
    # Extract weights and biases for the first layer
    nn_model.fc1.weight.data = torch.tensor(params[start:start + 1 * num_hidden].reshape(1, num_hidden), dtype=torch.float32)
    start += 1 * num_hidden
    nn_model.fc1.bias.data = torch.tensor(params[start:start + num_hidden], dtype=torch.float32)
    start += num_hidden
    
    # Extract weights and biases for the second layer
    nn_model.fc2.weight.data = torch.tensor(params[start:start + num_hidden * 1].reshape(num_hidden, 1), dtype=torch.float32)
    start += num_hidden * 1
    nn_model.fc2.bias.data = torch.tensor(params[start:start + 1], dtype=torch.float32)

# Initial guess for optimization
initial_guess = flatten_params(nn)

# Define bounds for parameters (for Differential Evolution)
bounds = [(-5, 5)] * len(initial_guess)

# Optimize the parameters using Nelder-Mead
result_nm = opt.minimize(nn_objective_function, initial_guess, args=(x_tensor, y_tensor, nn), method='Nelder-Mead')

# Optimize the parameters using Differential Evolution
result_de = differential_evolution(nn_objective_function, bounds, args=(x_tensor, y_tensor, nn), popsize=3, strategy='best1exp',
                                     maxiter=1000, vectorized=False, disp=False, polish=False)

# Define the BFGS optimization function
def bfgs_optimize(nn, x, y):
    """Optimize using BFGS."""
    def objective_function(params):
        return nn_objective_function(params, x, y, nn)
    
    result_bfgs = minimize(objective_function, flatten_params(nn), method='BFGS', options={'disp': True})
    return result_bfgs

# Optimize the parameters using BFGS
result_bfgs = bfgs_optimize(nn, x_tensor, y_tensor)

# Optimize the parameters using Adam (PyTorch optimizer)
def train_with_adam(nn, x, y, num_epochs=1000, learning_rate=0.01):
    """Train the neural network using Adam optimizer."""
    criterion = nn.MSELoss()
    optimizer = optim.Adam(nn.parameters(), lr=learning_rate)
    
    for epoch in range(num_epochs):
        nn.train()
        optimizer.zero_grad()
        y_pred = nn(x)
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()
    
    return nn

# Train the model using Adam optimizer
nn_adam = SimpleNN(num_inputs=1, num_hidden=num_hidden, num_outputs=1)
trained_nn_adam = train_with_adam(nn_adam, x_tensor, y_tensor)

# Predict with the optimized parameters and plot the results
def predict_with_nn(nn_model, x):
    """Predict with the neural network."""
    with torch.no_grad():
        nn_model.eval()
        y_pred = nn_model(torch.tensor(x, dtype=torch.float32).unsqueeze(1)).numpy()
    return y_pred

# Update network parameters and predict using the results from optimization
unflatten_params(nn, result_nm.x)
y_pred_nm = predict_with_nn(nn, x_data)

unflatten_params(nn, result_de.x)
y_pred_de = predict_with_nn(nn, x_data)

unflatten_params(nn, result_bfgs.x)
y_pred_bfgs = predict_with_nn(nn, x_data)

# Adam predictions
y_pred_adam = predict_with_nn(trained_nn_adam, x_data)

# Plot the results
plt.figure(figsize=(12, 8))
plt.plot(x_data, y_data, 'bo', label='True Data')
plt.plot(x_data, y_pred_nm, 'r-', label='Nelder-Mead Fit')
plt.plot(x_data, y_pred_de, 'g-', label='Differential Evolution Fit')
plt.plot(x_data, y_pred_bfgs, 'm-', label='BFGS Fit')
plt.plot(x_data, y_pred_adam, 'c-', label='Adam Fit')
plt.legend()
plt.xlabel('x')
plt.ylabel('y')
plt.title('Neural Network Fit to Polynomial Data')
plt.show()

# Print the results
print("\nOptimization Results:")
print("Nelder-Mead Objective:", result_nm.fun)
print("Differential Evolution Objective:", result_de.fun)
print("BFGS Objective:", result_bfgs.fun)


# CEC 2011 Problem

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.spatial.distance import pdist, squareform

def func_1(x):
    d = len(x)
    if d < 6:
        print('dimension-size should be six.')
        return None
    elif d > 6:
        print('dimension-size is more than 6.')
        print('function has been evaluated on first six dimensions.')
        x = x[:, :6]
    
    theta = 2 * np.pi / 100
    f = 0
    for t in range(101):
        y_t = x[0] * np.sin(x[1] * t * theta + x[2] * np.sin(x[3] * t * theta + x[4] * np.sin(x[5] * t * theta)))
        y_0_t = np.sin(5 * t * theta - 1.5 * np.sin(4.8 * t * theta + 2 * np.sin(4.9 * t * theta)))
        f += (y_t - y_0_t) ** 2
    
    return f

def func_2(x):
    d = len(x)
    if d % 3 != 0:
        print('x passed to this function must be n dimensional array where n is perfectly divisible by 3.')
        return None
    
    n = d // 3
    x = np.array(x).reshape(-1, 3)
    r = squareform(pdist(x))
    a = np.ones((n, n))
    b = np.full((n, n), 2)
    v = 0
    for i in range(n-1):
        for j in range(i+1, n):
            v += (a[i, j] / r[i, j]**12) - (b[i, j] / r[i, j]**6)
    
    return v

def func_5 (x) :
    d = len(x)
    if d % 3 != 0:
        print('x passed to this function must be n dimensional array where n is perfectly divisible by 3.')
        return None
    
    NP = d // 3
    x = np.array(x).reshape(-1, 3)
    R1 = 3.0
    R2 = 0.2
    A = 3.2647e+3
    B = 9.5373e+1
    lemda1 = 3.2394
    lemda2 = 1.3258
    lemda3 = 1.3258
    c = 4.8381
    d = 2.0417
    n1 = 22.956
    gama = 0.33675
    h = 0
    E = np.zeros(NP)
    r = np.zeros((NP, NP))
    fcr = np.zeros((NP, NP))
    VRr = np.zeros((NP, NP))
    VAr = np.zeros((NP, NP))

    for i in range(NP):
        for j in range(NP):
            r[i, j] = np.linalg.norm(x[i] - x[j])
            if r[i, j] < (R1 - R2):
                fcr[i, j] = 1
            elif r[i, j] > (R1 + R2):
                fcr[i, j] = 0
            else:
                fcr[i, j] = 0.5 - 0.5 * np.sin(np.pi/2 * (r[i, j] - R1) / R2)

            VRr[i, j] = A * np.exp(-lemda1 * r[i, j])
            VAr[i, j] = B * np.exp(-lemda2 * r[i, j])
    
    for i in range(NP):
        for j in range(NP):
            if i == j:
                continue
            jeta = np.zeros((NP, NP))
            for k in range(NP):
                if i == k or j == k:
                    continue
                rd1 = np.linalg.norm(x[i] - x[k])
                rd3 = np.linalg.norm(x[k] - x[j])
                rd2 = np.linalg.norm(x[i] - x[j])
                ctheta_ijk = (rd1**2 + rd2**2 - rd3**3) / (2 * rd1 * rd2)
                G_th_ijk = 1 + (c**2) / (d**2) - (c**2) / (d**2 + (h - ctheta_ijk)**2)
                jeta[i, j] += fcr[i, k] * G_th_ijk * np.exp(lemda3**3 * (r[i, j] - r[i, k])**3)
            
            Bij = (1 + (gama * jeta[i, j])**n1)**(-0.5/n1)
            E[i] += fcr[i, j] * (VRr[i, j] - Bij * VAr[i, j]) / 2
    
    return np.sum(E)

def func_6 (x) :
    d = len(x)
    if d % 3 != 0:
        print('x passed to this function must be n dimensional array where n is perfectly divisible by 3.')
        return None
    
    NP = d // 3
    x = np.array(x).reshape(-1, 3)
    R1 = 2.85
    R2 = 0.15
    A = 1.8308e+3
    B = 4.7118e+2
    lemda1 = 2.4799
    lemda2 = 1.7322
    lemda3 = 1.7322
    c = 1.0039e+05
    d = 1.6218e+01
    n1 = 7.8734e-01
    gama = 1.0999e-06
    h = -5.9826e-01
    E = np.zeros(NP)
    r = np.zeros((NP, NP))
    fcr = np.zeros((NP, NP))
    VRr = np.zeros((NP, NP))
    VAr = np.zeros((NP, NP))

    for i in range(NP):
        for j in range(NP):
            r[i, j] = np.linalg.norm(x[i] - x[j])
            if r[i, j] < (R1 - R2):
                fcr[i, j] = 1
            elif r[i, j] > (R1 + R2):
                fcr[i, j] = 0
            else:
                fcr[i, j] = 0.5 - 0.5 * np.sin(np.pi/2 * (r[i, j] - R1) / R2)

            VRr[i, j] = A * np.exp(-lemda1 * r[i, j])
            VAr[i, j] = B * np.exp(-lemda2 * r[i, j])
    
    for i in range(NP):
        for j in range(NP):
            if i == j:
                continue
            jeta = np.zeros((NP, NP))
            for k in range(NP):
                if i == k or j == k:
                    continue
                rd1 = np.linalg.norm(x[i] - x[k])
                rd3 = np.linalg.norm(x[k] - x[j])
                rd2 = np.linalg.norm(x[i] - x[j])
                ctheta_ijk = (rd1**2 + rd2**2 - rd3**3) / (2 * rd1 * rd2)
                G_th_ijk = 1 + (c**2) / (d**2) - (c**2) / (d**2 + (h - ctheta_ijk)**2)
                jeta[i, j] += fcr[i, k] * G_th_ijk * np.exp(lemda3**3 * (r[i, j] - r[i, k])**3)
            
            Bij = (1 + (gama * jeta[i, j])**n1)**(-0.5/n1)
            E[i] += fcr[i, j] * (VRr[i, j] - Bij * VAr[i, j]) / 2
    
    return np.sum(E)

In [ ]:
bounds_1 = [(-6.4, 6.35)]*6

lb_2 = [ 0., 0.,  0.,   -4.,   -4., -4. ,  -4. ,  -4.25, -4.25, -4.25, -4.5 , -4.5,
 -4.5,  -4.75, -4.75, -4.75, -5.,   -5.,  -5.,  -5.25, -5.25, -5.25, -5.5,  -5.5,
 -5.5, -5.75, -5.75, -5.75, -6.,  -6.  ]
ub_2 = [4.,  4.  ,       3.14159265 , 4. ,        4.  ,       4.,
 4.   , 4.25   ,    4.25   ,    4.25 ,      4.5    ,    4.5,
 4.5 ,       4.75,       4.75 ,      4.75,       5.   ,      5.,
 5.  ,       5.25  ,     5.25 ,      5.25,       5.5 ,     5.5,
 5.5 ,       5.75 ,      5.75 ,      5.75 ,      6. ,        6.        ]

bounds_2 = list(zip(lb_2, ub_2))
bounds_5 = [(0.0, 4.0),
 (0.0, 4.0),
 (0.0, 3.141592653589793),
 (-1.0, 4.0),
 (-1.0, 4.0),
 (-1.0, 4.0),
 (-1.0, 4.0),
 (-1.0, 4.25),
 (-1.0, 4.25),
 (-1.0, 4.25),
 (-1.0, 4.5),
 (-1.0, 4.5),
 (-1.0, 4.5),
 (-1.0, 4.75),
 (-1.0, 4.75),
 (-1.0, 4.75),
 (-1.0, 5.0),
 (-1.0, 5.0),
 (-1.0, 5.0),
 (-1.0, 5.25),
 (-1.0, 5.25),
 (-1.0, 5.25),
 (-1.0, 5.5),
 (-1.0, 5.5),
 (-1.0, 5.5),
 (-1.0, 5.75),
 (-1.0, 5.75),
 (-1.0, 5.75),
 (-1.0, 6.0),
 (-1.0, 6.0)]

dimension_1 = 6
dimension_2 = 30 
dimension_5=30 
dimension_6 = 30

In [ ]:
dimension=dimension_1
# Step 3: Define the objective function to minimize
def objective_function(x):
    """Objective function for optimization: Sum of squared errors."""
    return func_1(x)

executor = concurrent.futures.ThreadPoolExecutor(max_workers=10)
def objective_function_vect(xs, data):
    ret = list(executor.map(objective_function, xs))
    return ret

bounds=  bounds_1
Nmaxeval=50000#3000*dimension

# Optimize the coefficients using Nelder-Mead
initial_guess = [1.0 for i in range(dimension)]

#result =minimize(objective_function, initial_guess, args=(), method='Nelder-Mead', options={"maxfev":Nmaxeval, "adaptive":True }  ) 
#print("SIMPLEX done")
result_arrde = ARRDE(objective_function_vect, bounds, data=None,  x0=None, population_size=0, maxevals=Nmaxeval, tol=0.0 ).optimize()
print("Obj ARRDE :  ", result_arrde.fun)
result_lsrtde = LSRTDE(objective_function_vect, bounds, data=None,  x0=None, populationSize=0, maxevals=Nmaxeval ).optimize()
print("Obj LSRTDE :  ", result_lsrtde.fun)
#result_lshade = LSHADE(objective_function_vect, bounds, data=None,  x0=None, population_size=0, maxevals=Nmaxeval, options= {}).optimize()
#print("LSHADE done")
result_nlshadersp = NLSHADE_RSP (objective_function_vect, bounds, data=None, x0=None, population_size=0, 
                     maxevals=Nmaxeval, callback=None, seed=None, memory_size=20*dimension, archive_size_ratio=2.1).optimize()
print("Obj NLSHADE RSP :  ", result_nlshadersp.fun)
result_j20 = j2020(objective_function_vect, bounds, data=None,  x0=None, populationSize=0, maxevals=Nmaxeval ).optimize()
print("Obj j20 :  ", result_j20.fun)
#result_de = differential_evolution(objective_function, bounds, args=(), popsize=1, strategy='best1bin',
#                                         maxiter=int(Nmaxeval/(3*dimension)), vectorized=False, disp=False,polish=False)
#print("DE done")


print("")

#print("Obj NM :  ",result.fun)



#print("Obj lshade:  ", result_lshade.fun)

#print("Obj de :  ", result_de.fun)
executor.shutdown(wait=True)